# Panorama Oferta Formativa 2025 — Región de Los Ríos

Este cuaderno utiliza los datos limpios generados por el pipeline ETL para explorar la oferta de programas disponibles en la Región de Los Ríos. Las visualizaciones son interactivas (Plotly) y se enfocan en vacantes, costos y distribución por áreas del conocimiento.

In [4]:
from pathlib import Path
import unicodedata

import pandas as pd
import plotly.express as px

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_PATH = PROJECT_ROOT / "data/processed/oferta_formativa_clean.parquet"
if not DATA_PATH.exists():
    raise FileNotFoundError(f"No se encontró el archivo esperado: {DATA_PATH}")

df = pd.read_parquet(DATA_PATH)
df.shape

(3328, 57)

In [5]:
def normalize_text(value: str) -> str:
    if not isinstance(value, str):
        return ""
    normalized = unicodedata.normalize("NFKD", value)
    return ''.join(char for char in normalized if not unicodedata.combining(char)).lower()

region_mask = df['region_sede'].apply(normalize_text).str.contains('los rios', na=False)
los_rios = df.loc[region_mask].copy()

resumen = los_rios.agg({
    'codigo_carrera': 'nunique',
    'nombre_ies': 'nunique',
    'total_vacantes': 'sum',
    'arancel_anual': 'median',
    'matricula_anual': 'median'
})
resumen.rename({
    'codigo_carrera': 'programas_unicos',
    'nombre_ies': 'instituciones',
    'total_vacantes': 'vacantes_totales',
    'arancel_anual': 'arancel_median',
    'matricula_anual': 'matricula_median'
}, inplace=True)
resumen

programas_unicos        236.0
instituciones             9.0
vacantes_totales       9877.0
arancel_median      2902000.0
matricula_median     250000.0
dtype: float64

## 1. Programas con mayor oferta de vacantes
Las 15 carreras con más vacantes totales permiten dimensionar la capacidad disponible en la región y detectar concentración por institución.

In [6]:
top_vacantes = (
    los_rios.groupby(['nombre_carrera', 'nombre_ies'], as_index=False)
    .agg(total_vacantes=('total_vacantes', 'sum'),
         arancel_promedio=('arancel_anual', 'mean'))
    .sort_values('total_vacantes', ascending=False)
    .head(15)
)
fig_top = px.bar(
    top_vacantes,
    x='total_vacantes',
    y='nombre_carrera',
    color='nombre_ies',
    text='total_vacantes',
    orientation='h',
    title='Vacantes totales por carrera e institución (Top 15)',
    labels={'total_vacantes': 'Vacantes', 'nombre_carrera': 'Carrera', 'nombre_ies': 'IES'}
)
fig_top.update_layout(yaxis={'categoryorder': 'total ascending'}, height=700)
fig_top

## 2. Distribución de arancel anual por área
Comparativa de costos para evaluar diferencias de precio entre áreas del conocimiento y su potencial impacto en la accesibilidad.

In [7]:
fig_box = px.box(
    los_rios,
    x='area_conocimiento',
    y='arancel_anual',
    color='area_conocimiento',
    points='all',
    title='Distribución de arancel anual por área de conocimiento',
    labels={'area_conocimiento': 'Área', 'arancel_anual': 'Arancel anual (CLP)'}
)
fig_box.update_layout(xaxis_tickangle=-30, height=600, showlegend=False)
fig_box

## 3. Matrícula vs. arancel — tamaño según vacantes
Analizamos cómo se posicionan los programas en términos de costos de matrícula y arancel anual, ponderando la magnitud de vacantes ofertadas.

In [8]:
fig_scatter = px.scatter(
    los_rios,
    x='matricula_anual',
    y='arancel_anual',
    color='nombre_ies',
    size='total_vacantes',
    hover_name='nombre_carrera',
    title='Matrícula vs. Arancel anual (burbuja = vacantes)',
    labels={'matricula_anual': 'Matrícula (CLP)', 'arancel_anual': 'Arancel (CLP)'},
    height=650,
    size_max=32
)
fig_scatter